# 🚀 Argentum Overview - Cost-First AI Agent Platform

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarsZDF/argentum/blob/main/notebooks/01_argentum_overview.ipynb)

**Argentum** is a cost-first platform for building, debugging, and optimizing AI agent systems. This notebook provides a comprehensive introduction to all core components.

## 📋 What You'll Learn
- Installing and setting up Argentum
- Cost tracking and budget management
- State debugging with StateDiff
- Memory management with ContextDecay
- Agent coordination with Handoff
- Real-time cost alerts

⚡ **Zero setup required** - Everything installs automatically in this notebook!

In [ ]:
# Install Argentum if needed
try:
    import argentum
    print(f"✅ Argentum v{argentum.__version__} ready")
    
    # Check dependencies
    deps = argentum.check_dependencies()
    print(f"📊 Dependencies: {deps}")
    
    # Test cost optimization availability
    if deps.get('cost_optimization', False):
        print("✅ Cost optimization available")
    else:
        print("⚠️ Cost optimization not available - will use session-based approach")
        
except ImportError:
    print("📦 Installing Argentum...")
    !pip install --force-reinstall git+https://github.com/MarsZDF/argentum.git
    # Clear import cache
    import sys
    if 'argentum' in sys.modules:
        del sys.modules['argentum']
    
    import argentum
    print(f"✅ Argentum v{argentum.__version__} installed")
    
    # Check what's available after install
    deps = argentum.check_dependencies()
    print(f"📊 Available features: {deps}")

## 💰 Cost Tracking - The Heart of Argentum

In [ ]:
# Cost Tracking - Use the recommended agent session approach
import argentum

# Create an agent session to get the properly configured cost tracker
temp_session = argentum.create_agent_session(
    agent_id="cost_demo",
    half_life_steps=20,
    secure=True
)

# Get the cost tracker from the session (this ensures it has all methods)
tracker = temp_session["cost_tracker"]

print(f"💰 Using CostTracker from agent session: {type(tracker)}")
print(f"🔧 Available methods: {[m for m in dir(tracker) if not m.startswith('_')]}")

# Track some example API calls
tracker.record_usage(
    operation="completion",
    tokens_used=1500,
    agent_id="research_agent",
    model="gpt-4"
)

tracker.record_usage(
    operation="embedding",
    tokens_used=3000,
    agent_id="search_agent",
    model="text-embedding-3-small",
    cost=0.006
)

# Get cost report
report = tracker.get_cost_report()
print(f"💰 Total Cost: ${report.total_cost:.4f}")
print(f"🎯 Total Tokens: {report.total_tokens:,}")
print(f"🤖 Agent Breakdown:")
for agent, cost in report.breakdown.by_agent.items():
    print(f"  - {agent}: ${cost:.4f}")

## 🔍 State Debugging with StateDiff

In [ ]:
from argentum import StateDiff

# Initialize state tracking
diff = StateDiff()

# Simulate an agent working on a research task
initial_state = {
    "task": "research",
    "progress": 0.0,
    "findings": [],
    "confidence": 0.5,
    "metadata": {"sources_checked": 0, "quality_score": 0.0}
}

# Snapshot the initial state
diff.snapshot("start", initial_state)
print("📸 Initial state captured")

# Simulate progress
progress_state = initial_state.copy()
progress_state.update({
    "progress": 0.6,
    "findings": ["Key insight about cost optimization", "Performance bottleneck identified"],
    "confidence": 0.8,
    "metadata": {"sources_checked": 5, "quality_score": 0.85}
})

diff.snapshot("progress", progress_state)
print("📸 Progress state captured")

# Analyze the changes
changes = diff.diff("start", "progress")
print("\n🔍 State Changes Detected:")
for change in changes:
    print(f"  📝 {change['operation'].upper()}: {change['path']} = {change.get('value', 'N/A')}")

## 🧠 Memory Management with ContextDecay

In [ ]:
from argentum import ContextDecay

# Initialize context with 5-step half-life
context = ContextDecay(half_life_steps=5)

# Add some conversation history
context.add("user_question_1", "How do I optimize my AI costs?", importance=0.9)
context.add("response_1", "Use Argentum for cost tracking and optimization", importance=0.8)
context.add("user_question_2", "What's the weather like?", importance=0.3)
context.add("user_question_3", "Can you explain the StateDiff component?", importance=0.9)

print("📚 Initial context items:")
for key, item in context.items.items():
    print(f"  - {key}: importance={item.importance:.2f}, weight={item.current_weight:.2f}")

# Simulate time passing (conversation continues)
print("\n⏰ Time passing (3 steps)...")
for step in range(3):
    context.step()
    print(f"  Step {step + 1}: {len(context.get_active())} active items")

print("\n📚 Active context after decay:")
active_items = context.get_active()
for key, item in active_items.items():
    print(f"  - {key}: weight={item.current_weight:.2f} (importance={item.importance:.2f})")

## 🤝 Agent Coordination with Handoff

In [ ]:
from argentum import Handoff

# Create a handoff from research agent to writing agent
handoff = Handoff.create(
    from_agent="research_agent",
    to_agent="writing_agent", 
    context_summary="Research completed on cost optimization strategies. Found 3 key approaches.",
    artifacts=[
        "cost_analysis_data.json",
        "optimization_recommendations.md"
    ],
    confidence=0.85,
    metadata={
        "research_duration": "2.5 hours",
        "sources_consulted": 12,
        "cost_savings_potential": "30-40%"
    }
)

print("🤝 Handoff Created:")
print(f"  📋 ID: {handoff.handoff_id}")
print(f"  🔄 {handoff.from_agent} → {handoff.to_agent}")
print(f"  📝 Context: {handoff.context_summary}")
print(f"  📊 Confidence: {handoff.confidence}")
print(f"  📎 Artifacts: {len(handoff.artifacts)} files")

# Serialize for network transfer
handoff_json = handoff.to_json()
print(f"\n📡 Serialized handoff: {len(handoff_json)} characters")

# Receiving agent can recreate the handoff
received_handoff = Handoff.from_json(handoff_json)
print(f"✅ Handoff received by {received_handoff.to_agent}")

# Create receipt for acknowledgment
receipt = received_handoff.create_receipt(
    status="accepted",
    notes="All artifacts received. Beginning writing phase."
)
print(f"📧 Receipt created: {receipt['status']}")

## 🚨 Cost Alerts Setup

In [ ]:
from argentum import CostAlerts

# Initialize cost alerts
alerts = CostAlerts()

# Add budget alerts
dev_alert = alerts.add_budget_alert(
    threshold_dollars=25.0,
    email="dev-team@company.com",
    message_template="💰 Development budget alert: ${cost} spent by {agent_id}"
)

prod_alert = alerts.add_budget_alert(
    threshold_dollars=100.0,
    email="ops-team@company.com",
    message_template="🚨 Production budget exceeded: ${cost} > ${threshold}",
    severity="high"
)

# Add token usage alert
token_alert = alerts.add_token_alert(
    threshold_tokens=50000,
    email="ai-team@company.com",
    message_template="🎯 High token usage: {tokens_used} tokens for {agent_id}"
)

print(f"🚨 Cost Alerts Configured:")
print(f"  📊 Budget alerts: {len([r for r in alerts.rules if r.rule_type == 'budget'])}")
print(f"  🎯 Token alerts: {len([r for r in alerts.rules if r.rule_type == 'token'])}")
print(f"  📧 Total rules: {len(alerts.rules)}")

# Test alert triggering (simulation)
print("\n🧪 Testing alert triggers:")
test_scenarios = [
    ("normal_agent", 15.0, 12000, "Normal usage"),
    ("heavy_agent", 35.0, 45000, "Heavy usage - budget alert"),
    ("runaway_agent", 120.0, 75000, "Runaway agent - multiple alerts")
]

for agent_id, cost, tokens, description in test_scenarios:
    triggered_count = 0
    for rule in alerts.rules:
        if rule.rule_type == 'budget' and cost >= rule.threshold_dollars:
            triggered_count += 1
        elif rule.rule_type == 'token' and tokens >= rule.threshold_tokens:
            triggered_count += 1
    
    status = f"🚨 {triggered_count} alerts" if triggered_count > 0 else "✅ No alerts"
    print(f"  - {description}: {status}")

## 📊 Cost Export and Reporting

In [ ]:
from argentum import CostExporter
import json

# Create cost exporter
exporter = CostExporter()

# Add some sample cost data
sample_data = [
    {"agent_id": "research_agent", "cost": 15.50, "tokens": 12000, "model": "gpt-4"},
    {"agent_id": "writing_agent", "cost": 8.25, "tokens": 8500, "model": "gpt-3.5-turbo"},
    {"agent_id": "review_agent", "cost": 4.75, "tokens": 3200, "model": "gpt-4"},
]

# Export to different formats (simulated)
print("📊 Cost Export Capabilities:")
print("  📄 CSV: Spreadsheet-friendly format for analysis")
print("  📊 Excel: Multi-sheet reports with charts")
print("  📋 PDF: Executive summaries and reports")
print("  🔗 JSON: API integration and programmatic access")

# Generate summary report
total_cost = sum(item["cost"] for item in sample_data)
total_tokens = sum(item["tokens"] for item in sample_data)
agent_count = len(set(item["agent_id"] for item in sample_data))

print(f"\n📈 Sample Cost Summary:")
print(f"  💰 Total Cost: ${total_cost:.2f}")
print(f"  🎯 Total Tokens: {total_tokens:,}")
print(f"  🤖 Active Agents: {agent_count}")
print(f"  📊 Avg Cost/Token: ${total_cost/total_tokens:.6f}")

# Show breakdown by agent
print("\n🤖 Cost by Agent:")
for item in sorted(sample_data, key=lambda x: x['cost'], reverse=True):
    percentage = (item['cost'] / total_cost) * 100
    print(f"  - {item['agent_id']}: ${item['cost']:.2f} ({percentage:.1f}%)")

## 🎯 Complete Agent Session Example

In [ ]:
# Create a complete agent session with all components
session = argentum.create_agent_session(
    agent_id="demo_agent",
    half_life_steps=20,
    secure=True
)

# Extract components (handle None for backward compatibility)
tracker = session.get("cost_tracker")
context = session["context_decay"]
state_diff = session["state_diff"]
alerts = session.get("alerts")

print("🎯 Complete Agent Session Created:")
print(f"  💰 Cost tracking: {type(tracker).__name__ if tracker else 'Not available'}")
print(f"  🧠 Context management: {type(context).__name__}")
print(f"  🔍 State debugging: {type(state_diff).__name__}")
print(f"  🚨 Cost alerts: {type(alerts).__name__ if alerts else 'Not available'}")

# Set up basic cost protection if alerts available
if alerts:
    alerts.add_budget_alert(
        threshold_dollars=50.0,
        email="admin@company.com",
        message_template="Budget exceeded for demo_agent"
    )

# Simulate some agent work
print("\n🤖 Simulating agent work...")

# 1. Add context
context.add("task", "Demonstrate Argentum capabilities", importance=0.9)
context.add("method", "Interactive notebook tutorial", importance=0.8)

# 2. Track state
initial_state = {"step": 1, "status": "starting", "progress": 0.0}
state_diff.snapshot("init", initial_state)

# 3. Record costs if tracker available
if tracker:
    safe_record_usage(
        tracker=tracker,
        operation="tutorial",
        tokens_used=2500,
        agent_id="demo_agent",
        model="gpt-4"
    )

# 4. Progress and track changes
progress_state = {"step": 2, "status": "completed", "progress": 1.0}
state_diff.snapshot("complete", progress_state)

# Show results
if tracker:
    if hasattr(tracker, 'get_cost_report'):
        report = tracker.get_cost_report(agent_id="demo_agent")
    else:
        report = tracker.get_report()
changes = state_diff.diff("init", "complete")
active_context = context.get_active()

print(f"\n📊 Session Results:")
if tracker:
    print(f"  💰 Cost: ${report.total_cost:.4f}")
    print(f"  🎯 Tokens: {report.total_tokens:,}")
else:
    print(f"  💰 Cost tracking: Not available in this version")
print(f"  🔍 State changes: {len(changes)}")
print(f"  🧠 Active context items: {len(active_context)}")
if alerts:
    print(f"  🚨 Alert rules: {len(alerts.rules)}")

print("\n✅ Complete agent session demonstrated successfully!")

## 🏁 Summary & Next Steps

**Congratulations!** You've learned the core components of Argentum:

✅ **CostTracker** - Monitor and control AI spending  
✅ **StateDiff** - Debug agent behavior and state changes  
✅ **ContextDecay** - Manage agent memory efficiently  
✅ **Handoff** - Coordinate between multiple agents  
✅ **CostAlerts** - Get notified before costs spiral  
✅ **CostExporter** - Generate reports and analytics  

### 🚀 Next Steps:
1. **[State Debugging](02_state_diff.ipynb)** - Deep dive into StateDiff
2. **[Memory Management](03_context_decay.ipynb)** - Master ContextDecay
3. **[Cost Alerts](06_cost_alerts.ipynb)** - Production monitoring setup

### 🔧 Quick Integration:
```python
import argentum

# One-line setup for cost-aware agents
session = argentum.create_agent_session("my_agent")
tracker = session["cost_tracker"]

# Your agent code here...
tracker.record_usage(operation="completion", tokens_used=1500, agent_id="my_agent")
```

### 📖 Resources:
- **[GitHub Repository](https://github.com/MarsZDF/argentum)** - Source code and examples
- **[Agent Guide](../AGENT_GUIDE.md)** - Machine-readable integration guide
- **[API Documentation](https://argentum-agent.readthedocs.io)** - Complete reference

**Ready to build cost-optimized AI agents? 🚀💰**